In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import scipy

In [157]:
def generate_data(shift_amt=0.95, seed=15):
    num_train = 1000
    num_test = 1000
    total_points = num_train + num_test
    DATA_DIM = 2

    # Generate all data
    np.random.seed(seed)
    X = np.random.normal(loc=0, scale=1, size=(total_points, DATA_DIM))
    X = np.hstack((X, np.zeros(shape=(total_points, DATA_DIM))))
    theta = np.array([1 for _ in range(DATA_DIM * 2)])

    # Get training data
    X_train = X[:num_train]
    eps_train = np.random.normal(loc=0, scale=0.01, size=(num_train, 1))
    y_train = np.array([X_train[i].dot(theta) + eps_train[i] for i in range(len(X_train))])

    # Covariate shift for training data
    feature_idx = DATA_DIM - 1
    affected_idxs = set(np.random.choice(range(len(X_train)), int(len(X_train) * shift_amt), replace=False))
    # Compute mean for mean imputation
    feature_mean = 0
    for i in range(len(X_train)):
        if i not in affected_idxs:
            feature_mean += X_train[i][feature_idx]
    feature_mean /= (len(X_train) - len(affected_idxs))
    # Replace features with imputed mean and mark replacement in binary mask
    copy = list(X_train)
    for i in affected_idxs:
        copy[i][feature_idx] = feature_mean
        copy[i][-1] = 1
    X_train = np.array(copy)
    
    X_train = torch.tensor(X_train, dtype=torch.float)
    y_train = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)

    # Get test data
    X_test = X[num_train:]
    eps_test = np.random.normal(loc=0, scale=0.01, size=(num_test, 1))
    y_test = np.array([X_test[i].dot(theta) + eps_test[i] for i in range(len(X_test))])
    y_test = y_test.reshape(-1, 1)

    return X_train, X_test, y_train, y_test

In [158]:
X_train, X_test, y_train, y_test = generate_data()

In [159]:
import sys
sys.path.insert(0, '/Users/william/Documents/CMU/Research/RiskSensitiveLearning/Supervised Learning/objectives')
import cvar, human_aligned_risk, entropic_risk, trimmed_risk, mean_variance

In [160]:
input_size = 4
h1 = 10
h2 = 5
epochs = 200

class RegressionMLP(nn.Module):
    def __init__(self, input_size, h1, h2):
        super(RegressionMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [161]:
seeds = [15, 30]

objectives = {
    'Expected Value': nn.MSELoss(reduction='mean'),
    'CVaR': cvar.CVaR(a=0.9, criterion=nn.MSELoss(reduction='none')), # We look at bottom alpha% of losses
    'Entropic Risk': entropic_risk.EntropicRisk(t=-0.05, criterion=nn.MSELoss(reduction='none')),
    'Human-Aligned Risk': human_aligned_risk.HumanAlignedRisk(a=0.9, b=0.9, criterion=nn.MSELoss(reduction='none')),
    'Inverted CVaR': cvar.CVaR(a=0.95, inverted=True, criterion=nn.MSELoss(reduction='none')), # We look at top alpha% of losses
    'Mean-Variance': mean_variance.MeanVariance(c=0.05, criterion=nn.MSELoss(reduction='none')),
    'Trimmed Risk': trimmed_risk.TrimmedRisk(a=0.05, criterion=nn.MSELoss(reduction='none')),
}

for objective in objectives.keys():
    overall_risks = []

    for seed in seeds:
        X_train, X_test, y_train, y_test = generate_data(seed=seed)
        # Get objective
        criterion = objectives[objective]

        # Reset random seeds for comparaison.
        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)

        model = RegressionMLP(input_size, h1, h2)
        
        # Train model
        model.train()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        for epoch in range(epochs):
            # clear the gradients so they wont accumulate
            optimizer.zero_grad()

            output = model(X_train)
            loss = criterion(output.flatten(), y_train.flatten().type(torch.float32))

            # calculate gradient
            loss.backward()

            # gradient descent
            optimizer.step()

        # Evaluate model
        model.eval()
        overall_losses = []
        
        criterion = nn.MSELoss(reduction='mean')

        with torch.no_grad():
            for i, sample in enumerate(list(zip(X_test, y_test))):
                X, y = sample
                output = model(torch.tensor(X, dtype=torch.float32))

                loss = criterion(output, torch.tensor(y, dtype=torch.float32))
                
                overall_losses.append(loss)

        overall_risk = torch.mean(torch.tensor(overall_losses)).item()
        overall_risks.append(overall_risk)
    print('Objective: {}, Overall MSE: {}'.format(objective, np.mean(overall_risks)))

Objective: Expected Value, Overall MSE: 0.057154715061187744
Objective: CVaR, Overall MSE: 0.04862868785858154
Objective: Entropic Risk, Overall MSE: 0.05356391333043575
Objective: Human-Aligned Risk, Overall MSE: 0.053668634966015816
Objective: Inverted CVaR, Overall MSE: 0.056074295192956924
Objective: Mean-Variance, Overall MSE: 0.05394415557384491
Objective: Trimmed Risk, Overall MSE: 0.05284623242914677
